## HW#5 (For Number 1,2, and 4 "Bonus")

### Name : Tri Wanda Septian (106998406), Anggara Aji Saputra (106998412)

- Data: 
    - [Google web graph Data Set] from Stanford Large Network (SNAP) Dataset Collection
    - The data was released in 2002 by Google as a part of Google Programming Contest
    - Available at: http://snap.stanford.edu/data/web-Google.html 
- Format:
    - A text file containing around 5 million lines 
    - Each line is a directed edge representing hyperlinks between nodes (web pages)
    
    FromNodeId ToNodeId
    
    …



### Introduction
#### GraphFrames Overview

GraphFrames is a package for Apache Spark which provides DataFrame-based Graphs. It provides high-level APIs in Scala, Java, and Python. It aims to provide both the functionality of GraphX and extended functionality taking advantage of Spark DataFrames. This extended functionality includes motif finding, DataFrame-based serialization, and highly expressive graph queries.

#### What are GraphFrames?

GraphX is to RDDs as GraphFrames are to DataFrames.

GraphFrames represent graphs: vertices (e.g., users) and edges (e.g., relationships between users). If you are familiar with GraphX, then GraphFrames will be easy to learn. The key difference is that GraphFrames are based upon Spark DataFrames, rather than RDDs.

GraphFrames also provide powerful tools for running queries and standard graph algorithms. With GraphFrames, you can easily search for patterns within graphs, find important vertices, and more. Refer to the User Guide for a full list of queries and algorithms.

Will GraphFrames be part of Apache Spark?

The GraphX component of Apache Spark has no DataFrames- or Dataset-based equivalent, so it is natural to ask this question. The current plan is to keep GraphFrames separate from core Apache Spark for the time being:

- we are still considering making small adjustments to the API. The GraphFrames project will be considered for inclusion into Spark once we are confident that the current API addresses current and future needs.

- some important features present in GraphX such as partitioning are missing. We would like to offer some equivalent operations before considering merging with the Spark project.

- GraphFrames is used as a testbed for advanced, graph-specific optimizations into Spark’s Catalyst engine. Having them in a separate project accelerates the development cycle.

That being said, GraphFrames follows the same code quality standards as Spark, and it is cross-compiled and published for a large number of Spark versions. It is easy for users to depend on it.

In [1]:
#load graphframes
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = ('--jars graphframes-assembly-0.6.0-SNAPSHOT-spark2.3.jar pyspark-shell')

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType
from graphframes import *
from pyspark.sql.functions import desc

In [3]:
conf = SparkConf().setMaster("spark://sparklab1:7077").setAppName("HW#5")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext=SQLContext(sc)

In [4]:
sc

<SparkContext master=spark://sparklab1:7077 appName=PySparkShell>

In [5]:
raw_data = sc.textFile("/home/tri/Spark/dataset/web-Google-2.txt",use_unicode=False)

In [6]:
rdd = raw_data.map(lambda x:x.split("\t"))

In [7]:
rdd.take(5)

[['0', '11342'],
 ['0', '824020'],
 ['0', '867923'],
 ['0', '891835'],
 ['11342', '0']]

In [8]:
keys = rdd.flatMap(lambda x: (x[0], x[1])).distinct()
keylist = keys.collect()

#### Creating GraphFrames

Users can create GraphFrames from vertex and edge DataFrames.

- Vertex DataFrame: A vertex DataFrame should contain a special column named “id” which specifies unique IDs for each vertex in the graph.
- Edge DataFrame: An edge DataFrame should contain two special columns: “src” (source vertex ID of edge) and “dst” (destination vertex ID of edge).

Both DataFrames can have arbitrary other columns. Those columns can represent vertex and edge attributes.

A GraphFrame can also be constructed from a single DataFrame containing edge information. The vertices will be inferred from the sources and destinations of the edges.

In [9]:
vertices = keys.map(lambda x: (x, )).toDF(["id"])

In [10]:
edge = rdd.map(lambda x: (x[0], x[1])).toDF(["src", "dst"])

## (1) Given the Google web graph dataset, please output the list of web pages with the number of outlinks, sorted in descending order of the out-degrees.

#### Indegree and outdegree

For a vertex, the number of head ends adjacent to a vertex is called the indegree of the vertex and the number of tail ends adjacent to a vertex is its outdegree (called "branching factor" in trees).

Let G = (V, A) and v∈V. The indegree of v is denoted deg−(v) and its outdegree is denoted deg+(v).

A vertex with deg−(v) = 0 is called a source, as it is the origin of each of its outcoming arrows. Similarly, a vertex with deg+(v) = 0 is called a sink, since it is the end of each of its incoming arrows.

If a vertex is neither a source nor a sink, it is called an internal.[citation needed]
If for every vertex v∈V, deg+(v) = deg−(v), the graph is called a balanced directed graph

In [11]:
g = GraphFrame(vertices, edge)
g.outDegrees.withColumnRenamed("id","NodeId").sort(desc("outDegree")).show()

+------+---------+
|NodeId|outDegree|
+------+---------+
|506742|      456|
|203748|      372|
|305229|      372|
|768091|      330|
|808643|      277|
|412410|      268|
|600479|      265|
|376428|      258|
|156950|      257|
|885728|      256|
|667584|      253|
|685695|      248|
|282140|      247|
|598188|      245|
|579314|      244|
|411593|      231|
|321091|      229|
|838278|      225|
|302733|      216|
|915273|      213|
+------+---------+
only showing top 20 rows



In [12]:
#SaveFile to CSV
outDegreeResult = g.outDegrees.withColumnRenamed("id","NodeId").sort(desc("outDegree"))
outDegreeResult.coalesce(1).write.format("csv").options(header='true').save("/home/tri/Spark/ipnyb/output/hw#5/Q1/"+"outDegree")

## (2) Please output the inlink distribution of the top linked web pages, sorted in descending order of the in-degrees.

In [13]:
g.inDegrees.withColumnRenamed("id","NodeId").sort(desc("inDegree")).show()

+------+--------+
|NodeId|inDegree|
+------+--------+
|537039|    6326|
|597621|    5354|
|504140|    5271|
|751384|    5182|
| 32163|    5097|
|885605|    4847|
|163075|    4731|
|819223|    4620|
|605856|    4550|
|828963|    4484|
|551829|    4220|
| 41909|    4219|
|558791|    4206|
|459074|    4187|
|407610|    4180|
|213432|    4084|
|765334|    4015|
|384666|    4010|
|173976|    3988|
|687325|    3956|
+------+--------+
only showing top 20 rows



In [14]:
inDegreesResult = g.inDegrees.withColumnRenamed("id","NodeId").sort(desc("inDegree"))
inDegreesResult.coalesce(1).write.format("csv").options(header='true').save("/home/tri/Spark/ipnyb/output/hw#5/Q2/"+"inDegree")

## (3) Design an algorithm that maintains the connectivity of two nodes in an efficient way. Given a node v, please output the list of nodes that v points to, and the list of nodes that points to v. 

## (4) Compute the PageRank of the graph using MapReduce.

### PageRank

There are two implementations of PageRank.

- The first implementation uses the standalone GraphFrame interface and runs PageRank for a fixed number of iterations. This can be run by setting maxIter.

- The second implementation uses the org.apache.spark.graphx.Pregel interface and runs PageRank until convergence. This can be run by setting tol.

Both implementations support non-personalized and personalized PageRank, where setting a sourceId personalizes the results for that vertex.

In [15]:
## Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.15, maxIter=10)
results.vertices.select("id","pagerank").sort("pagerank",ascending=False).show()

+------+-----------------+
|    id|         pagerank|
+------+-----------------+
| 41909|812.1452403095795|
|597621|811.9656164662221|
|163075|799.5590536805101|
|537039|790.7848548128268|
|384666|691.5460438480312|
|504140| 673.807472416884|
|486980|653.0737401672387|
|605856|639.4883718897789|
|558791| 634.690891666088|
| 32163|628.9900597536641|
|551829|626.2736300380823|
|765334| 595.099059888169|
|751384|588.4882984660073|
|425770|552.1132327933444|
|908351|544.5875812030654|
|173976|543.1302882410182|
|  7314|531.2650816652405|
|213432|527.5442402285704|
|885605|521.1035964936184|
|691633|517.6216166344138|
+------+-----------------+
only showing top 20 rows



In [16]:
#SaveFile to CSV
PageRankResult = results.vertices.select("id","pagerank").sort("pagerank",ascending=False)
PageRankResult.coalesce(1).write.format("csv").options(header='true').save("/home/tri/Spark/ipnyb/output/hw#5/Q4/"+"PageRank")

### References :
1. Apache Spark GraphFrames - https://graphframes.github.io/quick-start.html